In [ ]:
!pip install cryptography

import os
import sqlite3
from cryptography.fernet import Fernet

def generate_key():
    key = Fernet.generate_key()
    with open("secret.key", "wb") as key_file:
        key_file.write(key)
    return key

def load_key():
    return open("secret.key", "rb").read()

if not os.path.exists("secret.key"):
    generate_key()
key = load_key()
cipher_suite = Fernet(key)

def initialize_db():
    conn = sqlite3.connect('passwords.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS passwords
                      (id INTEGER PRIMARY KEY, service TEXT, username TEXT, password TEXT)''')
    conn.commit()
    conn.close()

initialize_db()

def encrypt_password(password):
    return cipher_suite.encrypt(password.encode()).decode()

def decrypt_password(encrypted_password):
    return cipher_suite.decrypt(encrypted_password.encode()).decode()

import secrets
import string

def generate_password(length=12):
    characters = string.ascii_letters + string.digits + string.punctuation
    password = ''.join(secrets.choice(characters) for i in range(length))
    return password

def save_password(service, username, password):
    encrypted_password = encrypt_password(password)
    conn = sqlite3.connect('passwords.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO passwords (service, username, password) VALUES (?, ?, ?)",
                   (service, username, encrypted_password))
    conn.commit()
    conn.close()

def retrieve_password(service):
    conn = sqlite3.connect('passwords.db')
    cursor = conn.cursor()
    cursor.execute("SELECT username, password FROM passwords WHERE service=?", (service,))
    result = cursor.fetchone()
    conn.close()
    if result:
        username, encrypted_password = result
        password = decrypt_password(encrypted_password)
        return username, password
    return None

def text_based_interface():
    while True:
        print("\nPassword Manager")
        print("1. Generate Password")
        print("2. Save Password")
        print("3. Retrieve Password")
        print("4. Exit")
        choice = input("Enter choice: ")

        if choice == '1':
            length = int(input("Enter desired password length: "))
            password = generate_password(length)
            print(f"Generated Password: {password}")

        elif choice == '2':
            service = input("Enter service name: ")
            username = input("Enter username: ")
            password = input("Enter password: ")
            save_password(service, username, password)
            print("Password saved successfully.")

        elif choice == '3':
            service = input("Enter service name: ")
            result = retrieve_password(service)
            if result:
                username, password = result
                print(f"Username: {username}")
                print(f"Password: {password}")
            else:
                print("No password found for the given service.")

        elif choice == '4':
            break

        else:
            print("Invalid choice. Please try again.")

# Run the text-based interface
text_based_interface()



Password Manager
1. Generate Password
2. Save Password
3. Retrieve Password
4. Exit
Enter choice: 1
Enter desired password length: 8
Generated Password: =X$67s=_

Password Manager
1. Generate Password
2. Save Password
3. Retrieve Password
4. Exit
Enter choice: 1
Enter desired password length: 8
Generated Password: !UQ.CGfM

Password Manager
1. Generate Password
2. Save Password
3. Retrieve Password
4. Exit
